In [1]:
import torch
import torch.nn as nn
from PIL import Image
import numpy as np
import os
from torch.utils.data import Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
from torchvision.utils import save_image

In [3]:
import torch.optim as optim

In [4]:
class DiscBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel=4, stride=2):
        super(DiscBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,
                      out_channels=out_channels,
                      kernel_size=kernel,
                      stride=stride,
                      bias=False,
                      padding_mode='reflect'),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(negative_slope=0.2)
        )
    def forward(self, x):
        return self.conv(x)

In [5]:
class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=None, kernel=4, stride=2, padding=1):
        super().__init__()
        if not features:
            features = [64, 128, 256, 512]
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels*2, features[0], kernel_size=kernel, stride=stride, padding=padding, padding_mode='reflect'),
            nn.LeakyReLU(0.2)
        )

        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(DiscBlock(in_channels=in_channels,
                                   out_channels=feature,
                                   stride=1 if feature==features[-1] else 2,
                                   kernel=kernel
                                   ))
            in_channels = feature

        layers.append(nn.Conv2d(
            in_channels=in_channels,
            out_channels=1,
            kernel_size=kernel,
            stride = (1, 1),
            padding=1,
            padding_mode='reflect'
        ))
        self.model = nn.Sequential(*layers)

    def forward(self, x, y):
        x = torch.cat((x,y), dim=1)
        x = self.initial(x)
        x = self.model(x)
        return x

In [6]:
def test_discriminator():
    a = Discriminator()
    img = torch.ones(10, 3, 256, 256)
    print(a.forward(img, img).shape)
    # print(a.forward(img, img))

In [7]:
# test_discriminator()

In [8]:
class GeneratorBlock(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, act='relu', use_dropout=False):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,
                      out_channels=out_channels,
                      kernel_size=(4,4),
                      stride=(2,2),
                      padding=(1,1),
                      padding_mode='reflect')
            if down
            else nn.ConvTranspose2d(in_channels=in_channels,
                                    out_channels=out_channels,
                                    kernel_size=(4,4),
                                    stride=(2,2),
                                    padding=(1,1),
                                    bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act=='relu' else nn.LeakyReLU(0.2)
        )
        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.conv(x)
        x = self.dropout(x) if self.use_dropout else x
        return x

In [9]:
class Generator(nn.Module):
    def __init__(self, in_channels=3, features=64):
        super().__init__()
        self.initial_down = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=features,
                kernel_size=(4,4),
                stride=(2,2),
                padding=1,
                padding_mode='reflect'
            ),
            nn.LeakyReLU(0.2)
        )
        self.down1 = GeneratorBlock(in_channels=features, out_channels=features*2, down=True, act='leaky', use_dropout=False)
        self.down2 = GeneratorBlock(in_channels=features*2, out_channels=features*4, down=True, act='leaky', use_dropout=False)
        self.down3 = GeneratorBlock(in_channels=features*4, out_channels=features*8, down=True, act='leaky', use_dropout=False)
        self.down4 = GeneratorBlock(in_channels=features*8, out_channels=features*8, down=True, act='leaky', use_dropout=False)
        self.down5 = GeneratorBlock(in_channels=features*8, out_channels=features*8, down=True, act='leaky', use_dropout=False)
        self.down6 = GeneratorBlock(in_channels=features*8, out_channels=features*8, down=True, act='leaky', use_dropout=False)

        self.bottleneck = nn.Sequential(
            nn.Conv2d(in_channels=features*8,
                      out_channels=features*8,
                      kernel_size=(4,4), stride=(2,2),
                      padding=(1,1),
                      padding_mode="reflect"),
            nn.ReLU()
        )
        self.up1 = GeneratorBlock(in_channels=features*8, out_channels=features*8, down=False, act='relu', use_dropout=True)
        self.up2 = GeneratorBlock(in_channels=features*8*2, out_channels=features*8, down=False, act='relu', use_dropout=True)
        self.up3 = GeneratorBlock(in_channels=features*8*2, out_channels=features*8, down=False, act='relu', use_dropout=True)
        self.up4 = GeneratorBlock(in_channels=features*8*2, out_channels=features*8, down=False, act='relu', use_dropout=False)
        self.up5 = GeneratorBlock(in_channels=features*8*2, out_channels=features*4, down=False, act='relu', use_dropout=False)
        self.up6 = GeneratorBlock(in_channels=features*4*2, out_channels=features*2, down=False, act='relu', use_dropout=False)
        self.up7 = GeneratorBlock(in_channels=features*2*2, out_channels=features, down=False, act='relu', use_dropout=False)

        self.final_up = nn.Sequential(
            nn.ConvTranspose2d(in_channels=features*2, out_channels=in_channels,kernel_size=(4,4), stride=(2,2), padding=(1,1)),
            nn.Tanh()
        )

    def forward(self, x):
        d1 = self.initial_down(x)
        d2 = self.down1(d1)
        d3 = self.down2(d2)
        d4 = self.down3(d3)
        d5 = self.down4(d4)
        d6 = self.down5(d5)
        d7 = self.down6(d6)
        bottleneck = self.bottleneck(d7)
        up1 = self.up1(bottleneck)
        up2 = self.up2(torch.cat([d7, up1], dim=1))
        up3 = self.up3(torch.cat([d6, up2], dim=1))
        up4 = self.up4(torch.cat([d5, up3], dim=1))
        up5 = self.up5(torch.cat([d4, up4], dim=1))
        up6 = self.up6(torch.cat([d3, up5], dim=1))
        up7 = self.up7(torch.cat([d2, up6], dim=1))
        return self.final_up(torch.cat([d1, up7], dim=1))

In [10]:
def test_generator():
    test_imgs = torch.ones(10, 3, 256, 256)
    gen = Generator(3, 64)
    print(gen(test_imgs).shape)

In [11]:
# test_generator()

In [12]:
class MapDataset(Dataset):
    def __init__(self, dir, val=False):
        super().__init__()
        self.dir = dir
        self.all_files = os.listdir(dir)
        self.both_transform = A.Compose(
            [
                A.Resize(width=256, height=256),
                A.HorizontalFlip(p=0.5 if not val else 0)
            ],
            additional_targets={
                'image0': 'image'
            }
        )
        self.transform_in = A.Compose(
            transforms=[A.ColorJitter(p=0.2 if not val else 0),
                        A.Normalize(mean=[0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5]),
                        ToTensorV2()]
        )
        self.transform_out = A.Compose(
            transforms=[
                 A.Normalize(mean=[0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5]),
                 ToTensorV2()
            ]
        )
    def __len__(self):
        return len(self.all_files)
    def __getitem__(self, idx):
        img_path = os.path.join(self.dir, self.all_files[idx])
        image = np.array(Image.open(img_path))
        in_image = image[:, :600, :]
        out_image = image[:, 600:, :]
        augmentations = self.both_transform(image=in_image, image0=out_image)
        in_image, out_image = augmentations['image'], augmentations['image0']
        in_image = self.transform_in(image=in_image)['image']
        out_image = self.transform_out(image=out_image)['image']
        return in_image, out_image

In [13]:
dataset = MapDataset('maps/train', False)

In [14]:
# dataset[0][1].shape

In [15]:
def save_examples(generator, val_loader, epoch, folder):
    x, y = next(iter(val_loader))
    x, y = x.to(device), y.to(device)
    generator.eval()
    with torch.no_grad():
        y_fake = generator.forward(x)
        y_fake = y_fake*0.5+0.5
        save_image(y_fake, folder+f'/generated_{epoch}.png')
        save_image(x*0.5+0.5, folder+f'/input_{epoch}.png')
        save_image(y*0.5+0.5, folder+f'/real_{epoch}.png')
    generator.train(True)


In [16]:
def save_checkpoint(model, optimizer, filename):
    print('Saving checkpoint')
    checkpoint = {
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    torch.save(checkpoint, filename)

In [17]:
def load_checkpoint(model, optimizer, lr, file):
    print('load checkpoint')

    checkpoint = torch.load(file, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [18]:
lr = 2e-4
batch_size = 8
num_workers=1
num_epochs = 500
l1_lambda = 100
load_model = False
train_dataset = MapDataset(dir='maps/train')
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

g_scaler = torch.cuda.amp.GradScaler()
d_scaler = torch.cuda.amp.GradScaler()

val_dataset = MapDataset(dir='maps/val', val=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=1, shuffle=False)

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
discriminator = Discriminator().to(device)
generator = Generator().to(device)
optimizer_disc = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_gen = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
BCE = nn.BCEWithLogitsLoss()
l1_loss = nn.L1Loss()



if load_model:
    load_checkpoint(generator, optimizer_gen, lr=lr, file='generator_model.pth')
    load_checkpoint(discriminator, optimizer_disc, lr=lr, file='discriminator_model.pth')

for epoch in range(num_epochs):
    loop = tqdm(train_loader, leave=True)
    print(f'Обучение эпохи {epoch}/{num_epochs}')
    for idx, (x, y) in enumerate(loop):
        x = x.to(device)
        y = y.to(device)

        with torch.cuda.amp.autocast():
            y_fake = generator.forward(x)
            D_fake = discriminator.forward(x, y_fake.detach())
            D_real = discriminator.forward(x, y)
            # D_loss = BCE(D_real, torch.ones_like(D_real).to(device)) + BCE(D_fake, torch.zeros_like(D_fake).to(device))
            D_loss = BCE(D_real, torch.ones_like(D_real).to(device)) + BCE(D_fake, torch.zeros_like(D_fake).to(device))
        discriminator.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(optimizer_disc)
        d_scaler.update()

        with torch.cuda.amp.autocast():
            D_fake = discriminator.forward(x, y_fake)
            G_loss = BCE(D_fake, torch.ones_like(D_fake)) + l1_lambda * l1_loss(y_fake, y)

        generator.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(optimizer_gen)
        g_scaler.update()

    if epoch%5==0:
        save_checkpoint(generator, optimizer_gen, 'generator_model.pth')
        save_checkpoint(discriminator, optimizer_disc, 'discriminator_model.pth')

        save_examples(generator, val_loader, epoch, 'examples')

  0%|          | 0/137 [00:00<?, ?it/s]

Обучение эпохи 0/500


100%|██████████| 137/137 [02:10<00:00,  1.05it/s]


Saving checkpoint
Saving checkpoint


  0%|          | 0/137 [00:00<?, ?it/s]

Обучение эпохи 1/500


 91%|█████████ | 124/137 [01:58<00:12,  1.05it/s]


KeyboardInterrupt: 

In [23]:
generator.train()
save_examples(generator, val_loader, 111, 'examples')